In [ ]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

## Emissions, Population, GDP

In [ ]:
# import target, pop, and gdp data per state
y_df = pd.read_csv('data/climatewatch-usemissions.csv', usecols=['State', 'Year', 'Population (People)','State GDP (Million US$ (chained 1997/2005))','Transportation (MtCO2e)'])
y_df.head()

In [ ]:
y_df.count()

## Transit Ridership, Gasoline Usage, Vehicle Miles Traveled (VMT), Vehicles

In [ ]:
# import gas usage, transit ridership, vehicle miles traveled, and vehicle data per state
transit_df = pd.read_csv('data/transportation_usage.csv', encoding = 'utf-16', sep='\t')
transit_df.head()

In [ ]:
### Create a column for each measure in transit_df['Measures'] and assign the appropriate values
# Grab measure names
measures_list = transit_df['Measures'].unique()

# Limit list of measure names to gas usage, transit ridership, vehicle miles traveled, and vehicle data
measures_list = measures_list[0:4]

# Create initial dataframe based on transit ridership per state and per year
to_merge_df = transit_df.loc[transit_df['Measures']==measures_list[0]]
new_column_name = to_merge_df.iloc[0]['Measures']
to_merge_df.columns=['State', 'Measures', 'Year', new_column_name]
to_merge_df = to_merge_df.drop(columns=['Measures'])
transit_measures_df = to_merge_df

# Merge additional measures (gas usage, vehicle miles traveled, and vehicle data)
for column in measures_list[1:4]:
    to_merge_df = transit_df.loc[transit_df['Measures']== column]
    new_column_name = to_merge_df.iloc[0]['Measures']
    to_merge_df.columns=['State', 'Measures', 'Year', new_column_name]
    to_merge_df = to_merge_df.drop(columns=['Measures'])
    transit_measures_df = transit_measures_df.merge(to_merge_df, how = 'left', on = ['State', 'Year'])
transit_measures_df.head()

In [ ]:
# Remove null values (post csv exploration)
transit_measures_df = transit_measures_df.loc[transit_measures_df['Year'] >= 1994]
transit_measures_df = transit_measures_df.loc[transit_measures_df['State'] != 'Puerto Rico']
transit_measures_df.count()

In [ ]:
# merge y_df and transit_measures_df
final_df = y_df.merge(transit_measures_df, on = ['State', 'Year'])
final_df.head()

In [ ]:
final_df.count()

## Land Area (sq mi)

In [ ]:
# Scrape land area (in sq mi) from:
# https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area
wikiurl = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area#cite_note-2010census-2'

# Check response code to ensure ability to download
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

In [ ]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find('table',{'class':"wikitable"})
wiki_area_df = pd.read_html(str(indiatable))

# convert list to dataframe
wiki_area_df = pd.DataFrame(wiki_area_df[0])

# Drop unwanted level
wiki_area_df.columns = wiki_area_df.columns.droplevel(level=0)
wiki_area_df.head()

# Grab desired column (sq mi in index 5)
area_df = wiki_area_df.iloc[:, [0,5]]
area_df.head()

In [ ]:
# manually add a value for sq mi total for US
us_df = wiki_area_df.loc[wiki_area_df['State']=='50 states and District of Columbia']
us_sqmi_df = us_df.iloc[[0], [0, 5]]
us_sqmi_df['State'] = us_sqmi_df['State'].replace({'50 states and District of Columbia': 'United States'})
area_df = area_df.append(us_sqmi_df)
area_df

In [ ]:
final_df = final_df.merge(area_df, how='left', on = ['State'])
final_df

## Annual Temperature Highs

In [ ]:
# Grab maxTemp data from https://www.ncdc.noaa.gov/cag/ for each state
# Hawaii is notably omitted
# Citation: NOAA National Centers for Environmental information, Climate at a Glance: Statewide Time Series, published June 2021, retrieved on June 14, 2021

# Create dataframe to store values for all 50 states (except Hawaii)
ncdc_temp_df = pd.DataFrame(columns=['Year', 'tempHigh', 'State'])
for i in range(50):
    if i == 48:
        continue
    state_num = str(i+1)
    ncdc_url = 'https://www.ncdc.noaa.gov/cag/statewide/time-series/'+state_num+'-tmax-12-12-1990-2018.json?base_prd=true&begbaseyear=1925&endbaseyear=2000'
    response = requests.get(ncdc_url).json()

    # Get state name
    data_title = response['description']['title']
    state_name = data_title.split(',',1)[0]

    # Rename columns to year values (omitting '12' from end)
    ncdc_temp = pd.DataFrame(response['data'])
    ncdc_temp.columns = ncdc_temp.columns.str[0:4]

    # Convert to same row/column format as final_df
    ncdc_temp = ncdc_temp.transpose()

    # Remove anomoly and create new state column
    ncdc_temp = ncdc_temp.drop(columns='anomaly')
    ncdc_temp['State'] = state_name

    # Reset index and set new Year column to int instead of object
    ncdc_temp = ncdc_temp.reset_index()
    ncdc_temp = ncdc_temp.rename(columns={'index':'Year','value':'tempHigh'})
    ncdc_temp['Year'] = ncdc_temp['Year'].astype(int)
    ncdc_temp_df = ncdc_temp_df.append(ncdc_temp)

In [ ]:
# Grab national data
ncdc_url = 'https://www.ncdc.noaa.gov/cag/national/time-series/110-tmax-12-12-1990-2018.json?base_prd=true&begbaseyear=1925&endbaseyear=2000'
response = requests.get(ncdc_url).json()

# Get state name
state_name = 'United States'

# Rename columns to year values (omitting '12' from end)
ncdc_temp = pd.DataFrame(response['data'])
ncdc_temp.columns = ncdc_temp.columns.str[0:4]

# Convert to same row/column format as final_df
ncdc_temp = ncdc_temp.transpose()

# Remove anomoly and create new state column
ncdc_temp = ncdc_temp.drop(columns='anomaly')
ncdc_temp['State'] = state_name

# Reset index and set new Year column to int instead of object
ncdc_temp = ncdc_temp.reset_index()
ncdc_temp = ncdc_temp.rename(columns={'index':'Year','value':'tempHigh'})
ncdc_temp['Year'] = ncdc_temp['Year'].astype(int)
ncdc_temp_df = ncdc_temp_df.append(ncdc_temp)

In [ ]:
final_df = final_df.merge(ncdc_temp_df, how='left', on = ['State', 'Year'])
final_df

In [ ]:
# Export to CSV
final_df.to_csv("resources/annualAllStateData.csv", index = False, header = True)

## Number of Alternative Fueling Stations

In [ ]:
fuel_df = pd.read_csv('data/altFuelStations.csv', usecols=['Fuel Type Code', 'State', 'Open Date'])
fuel_df.head()

In [ ]:
# Convert Open Date to Year
fuel_df['Open Date'] = fuel_df['Open Date'].str[:4]

# Drop null values
fuel_df = fuel_df.dropna()

# Replace state abbreviations with full names
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

fuel_df['State'].replace(states, inplace=True)

In [ ]:
# Group by state and year to get counts of stations per state per year
fuel_df['stateYear'] = fuel_df['State'] + fuel_df['Open Date']
fuel_df
fuel_df_counts = fuel_df.groupby(['stateYear'])
fuel_df_counts = fuel_df_counts.count()
fuel_df_counts = fuel_df_counts.reset_index()
fuel_df_counts

In [ ]:
# Unpack state and year columns
fuel_df_counts['Year'] = fuel_df_counts['stateYear'].str[-4:]
fuel_df_counts['State'] = fuel_df_counts['stateYear'].str[:-4]

In [ ]:
# Create final counts dataframe for merge to final_df
fuel_df_counts_final = fuel_df_counts[['State', 'Year', 'Open Date']]
fuel_df_counts_final = fuel_df_counts_final.rename(columns={'Open Date':'altFuelStations'})
fuel_df_counts_final

In [ ]:
type(fuel_df_counts_final['State'])

In [ ]:
test_df = final_df.merge(fuel_df_counts_final, how='left', on = ['State', 'Year'])
test_df

In [ ]:
#final_df.count()